In [1]:
import tensorflow  as tf

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
v = tf.Variable(0.0)

In [4]:
(v + 1).numpy()

1.0

In [5]:
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [6]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [7]:
v.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6.0>

In [8]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>

In [9]:
v.read_value()

<tf.Tensor: id=22, shape=(), dtype=float32, numpy=6.0>

In [10]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as t:
    loss = w * w

In [11]:
grad = t.gradient(loss, w)

In [12]:
grad

<tf.Tensor: id=39, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [13]:
w = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(w)
    loss = w * w

In [14]:
dloss_dw = t.gradient(loss, w)

In [15]:
dloss_dw

<tf.Tensor: id=46, shape=(), dtype=float32, numpy=6.0>

In [16]:
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w * w
    z = y * y

In [17]:
dy_dw = t.gradient(y, w)

In [18]:
dy_dw

<tf.Tensor: id=54, shape=(), dtype=float32, numpy=6.0>

In [19]:
dz_dw = t.gradient(z, w)

In [20]:
dz_dw

<tf.Tensor: id=62, shape=(), dtype=float32, numpy=108.0>

In [21]:
(train_image, train_labels), _ = tf.keras.datasets.mnist.load_data()

In [22]:
train_image.shape

(60000, 28, 28)

In [23]:
train_image = tf.expand_dims(train_image, -1)

In [24]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [25]:
train_image = tf.cast(train_image/255, tf.float32)

In [26]:
train_labels = tf.cast(train_labels, tf.int64)

In [27]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))

In [28]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [29]:
dataset = dataset.shuffle(10000).batch(32)

In [30]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [32]:
optimizer = tf.keras.optimizers.Adam()

In [33]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
features, labels = next(iter(dataset))

In [35]:
features.shape

TensorShape([32, 28, 28, 1])

In [36]:
labels.shape

TensorShape([32])

In [37]:
predictions = model(features)

In [38]:
predictions.shape

TensorShape([32, 10])

In [39]:
tf.argmax(predictions, axis=1)

<tf.Tensor: id=197, shape=(32,), dtype=int64, numpy=
array([5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6,
       5, 6, 6, 6, 6, 6, 6, 6, 6, 6], dtype=int64)>

In [40]:
labels

<tf.Tensor: id=179, shape=(32,), dtype=int64, numpy=
array([2, 9, 7, 0, 3, 6, 3, 5, 6, 7, 1, 7, 6, 4, 6, 1, 0, 9, 1, 1, 2, 0,
       1, 2, 3, 6, 9, 2, 2, 2, 2, 7], dtype=int64)>

In [41]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [65]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [84]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [88]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('Epoch{} loss is {}'.format(epoch, train_loss.result()))
        print('Epoch{} Accuracy is {}'.format(epoch, train_accuracy.result()))
        train_loss.reset_states()
        train_accuracy.reset_states()

In [89]:
train()

Epoch0 loss is 0.9142864942550659
Epoch0 Accuracy is 0.7101799249649048
Epoch1 loss is 0.42674219608306885
Epoch1 Accuracy is 0.8613666892051697


KeyboardInterrupt: 

### tf.keras.metrics 汇总计算模块

In [57]:
m = tf.keras.metrics.Mean("loss")

In [58]:
m(10)

<tf.Tensor: id=342, shape=(), dtype=float32, numpy=10.0>

In [59]:
m(20)

<tf.Tensor: id=352, shape=(), dtype=float32, numpy=15.0>

In [60]:
m.result().numpy()

15.0

In [61]:
m([30, 40])

<tf.Tensor: id=366, shape=(), dtype=float32, numpy=25.0>

In [66]:
m.reset_states()

In [67]:
m(1)

<tf.Tensor: id=478, shape=(), dtype=float32, numpy=1.0>

In [68]:
m(2)

<tf.Tensor: id=488, shape=(), dtype=float32, numpy=1.5>

In [69]:
m.result()

<tf.Tensor: id=492, shape=(), dtype=float32, numpy=1.5>

In [62]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [63]:
a(labels, model(features))

<tf.Tensor: id=410, shape=(), dtype=float32, numpy=0.125>

In [86]:
train()

Epoch1874 is finshed
Epoch1874 is finshed
Epoch1874 is finshed
Epoch1874 is finshed
Epoch1874 is finshed


KeyboardInterrupt: 